# DFx Indicators ETL

## Database

This notebook is used to read ETL outputs and ingest the data into the database used by the DFx Indicators API.

### Libraries

In [1]:
import dfx_etl as dfx
import pandas as pd

### Configuration

In [2]:
# Set to the name of the versioned folder in the storage to pull the data from
DATA_VERSION = "v26-01-05"

engine = dfx.get_engine()
storage = dfx.get_storage()

`DB_CONN` is not configured. Using an in-memory SQLite database.


In [ ]:
# Does nothing when tables exist
# dfx.create_tables(engine)

['country', 'indicator', 'dimension', 'series']

### Data Preparation

In [4]:
# Read and concatenate all data
df_data = storage.read_dataset(DATA_VERSION)
print("Shape:", df_data.shape)
display(df_data.head())

Shape: (2881842, 7)


,indicator_name,country_code,year,value,provider,disaggregation,source
0,Proportion of population below international p...,ALB,2015,1.10000,unstats_sdg_database,BOTHSEX; ALLAGE; ALLAREA,<NA>
1,Proportion of population below international p...,ALB,2016,1.10000,unstats_sdg_database,BOTHSEX; ALLAGE; ALLAREA,<NA>
2,Proportion of population below international p...,ALB,2017,1.40000,unstats_sdg_database,BOTHSEX; ALLAGE; ALLAREA,<NA>
3,Proportion of population below international p...,ALB,2018,1.00000,unstats_sdg_database,BOTHSEX; ALLAGE; ALLAREA,<NA>
4,Proportion of population below international p...,ALB,2018,1.60093,unstats_sdg_database,BOTHSEX; <15Y; ALLAREA,<NA>


In [5]:
columns = {"indicator_name": "name", "provider": "provider"}
df_indicators = df_data.reindex(columns=columns).rename(columns=columns)
df_indicators.drop_duplicates(ignore_index=True, inplace=True)
df_indicators.sort_values(["provider", "name"], ignore_index=True, inplace=True)
print("Shape:", df_indicators.shape)
display(df_indicators.head())

Shape: (2170, 2)


,name,provider
0,(S.1) Extent to which countries have laws and ...,unstats_sdg_database
1,(S.1.C.1) Extent to which countries have laws ...,unstats_sdg_database
2,(S.1.C.2) Extent to which countries have laws ...,unstats_sdg_database
3,(S.1.C.3) Extent to which countries have laws ...,unstats_sdg_database
4,(S.1.C.4) Extent to which countries have laws ...,unstats_sdg_database


In [6]:
columns = {"disaggregation": "name"}
df_dimensions = df_data.reindex(columns=columns).rename(columns=columns)
df_dimensions.drop_duplicates(ignore_index=True, inplace=True)
df_dimensions.sort_values(list(columns.values()), ignore_index=True, inplace=True)
print("Shape:", df_dimensions.shape)
display(df_dimensions.head())

Shape: (12924, 1)


,name
0,1-14
1,1-4
2,10MBPS
3,15+; A
4,15-24


### Data Ingestion

In [7]:
# Ingest the data
df_indicators.to_sql(
    "indicator", con=engine, if_exists="append", index=False, method="multi"
)
df_dimensions.to_sql(
    "dimension", con=engine, if_exists="append", index=False, method="multi"
)

12924

In [8]:
df_countries = pd.read_sql(
    "SELECT id AS country_id, iso_3 AS country_code FROM country;", con=engine
)
df_indicators = pd.read_sql(
    "SELECT id AS indicator_id, name AS indicator_name FROM indicator;", con=engine
)
df_dimensions = pd.read_sql(
    "SELECT id AS dimension_id, name AS disaggregation FROM dimension;", con=engine
)

print("Shape before:", df_data.shape)
df_data = df_data.merge(df_countries, how="inner", on="country_code")
df_data = df_data.merge(df_indicators, how="inner", on="indicator_name")
df_data = df_data.merge(df_dimensions, how="inner", on="disaggregation")
df_data = df_data.reindex(
    columns=["country_id", "indicator_id", "dimension_id", "year", "value"]
)
df_data.sort_values(
    ["country_id", "indicator_id", "dimension_id", "year"], ignore_index=True
)
print("Shape after:", df_data.shape)
display(df_data.head())

Shape before: (2881842, 7)
Shape after: (2881842, 5)


,country_id,indicator_id,dimension_id,year,value
0,8,462,1587,2015,1.10000
1,8,462,1587,2016,1.10000
2,8,462,1587,2017,1.40000
3,8,462,1587,2018,1.00000
4,8,462,1519,2018,1.60093


In [9]:
df_data.to_sql(
    "series",
    con=engine,
    if_exists="append",
    index=False,
    chunksize=1024,  # Batch the writes
    method="multi",
)

2881842

In [10]:
# Sanity check
pd.read_sql(
    "SELECT COUNT(*), indicator_provider FROM observation GROUP BY indicator_provider;",
    con=engine,
)

,COUNT(*),indicator_provider
0,1149610,unstats_sdg_database
1,1732232,world_bank_wdi
